In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
import numpy as np
from math import sqrt, pi, log, e
from enum import Enum
import scipy.stats as stat
from scipy.stats import norm
from math import exp, sqrt

import time



In [ ]:
class BSMerton(object):
 

    def approximate_forward_difference(self, func, h=0.01):
        fx = func()
        fx_plus_h = func(h)
        return (fx_plus_h - fx) / h

    def approximate_backward_difference(self, func, h=0.01):
        fx = func()
        fx_minus_h = func(-h)
        return (fx - fx_minus_h) / h

    # Forward and backward difference approximations for Vega
    def vega_fd_approx(self, h=0.01):
        return self.approximate_forward_difference(self.vega, h)

    def vega_bd_approx(self, h=0.01):
        return self.approximate_backward_difference(self.vega, h)

    # Forward and backward difference approximations for Theta
    def theta_fd_approx(self, h=0.01):
        return self.approximate_forward_difference(self.theta, h)

    def theta_bd_approx(self, h=0.01):
        return self.approximate_backward_difference(self.theta, h)

    # Forward and backward difference approximations for Gamma
    def gamma_fd_approx(self, h=0.01):
        return self.approximate_forward_difference(self.gamma, h)

    def gamma_bd_approx(self, h=0.01):
        return self.approximate_backward_difference(self.gamma, h)


class BSMerton:
    # ... (previous code) ...

    def approximate_delta_fd(self, h=0.01):
        # Save the original underlying asset price
        original_S = self.S

        # Set the underlying asset price to the given value
        self.S += h

        # Calculate d1 and d2 with the new underlying asset price
        self.d1 = (log(self.S / self.K) + \
                   (self.r - self.q + 0.5 * (self.sigma ** 2)) \
                   * self.T) / self.sigmaT
        self.d2 = self.d1 - self.sigmaT

        # Calculate the option price
        option_price_plus_h = self.premium()[0]

        # Reset the underlying asset price to its original value
        self.S = original_S

        # Recalculate d1 and d2 with the original underlying asset price
        self.d1 = (log(self.S / self.K) + \
                   (self.r - self.q + 0.5 * (self.sigma ** 2)) \
                   * self.T) / self.sigmaT
        self.d2 = self.d1 - self.sigmaT

        # Calculate the option price
        option_price = self.premium()[0]

        # Calculate the approximate delta using forward differencing
        approximate_delta = (option_price_plus_h - option_price) / h

        return approximate_delta




In [47]:
class BSMerton(object):
    def __init__(self, args,b=None):
        self.Type = int(args[0])  # 1 for a Call, - 1 for a put
        self.S = float(args[1])  # Underlying asset price
        self.K = float(args[2])  # Option strike K
        self.r = float(args[3])  # Continuous risk fee rate
        self.q = float(args[4])  # Dividend continuous rate
        self.T = float(args[5]) / 365.0  # Compute time to expiry
        self.sigma = float(args[6])  # Underlying volatility
        self.sigmaT = self.sigma * self.T ** 0.5  # sigma*T for reusability
        self.d1 = (log(self.S / self.K) + \
                   (self.r - self.q + 0.5 * (self.sigma ** 2)) \
                   * self.T) / self.sigmaT
        self.d2 = self.d1 - self.sigmaT
        if b is None:
            self.b = self.r - self.q
        else:
            self.b = b

        [self.Premium] = self.premium()
        [self.Delta] = self.delta()
        [self.Theta] = self.theta()
        [self.Rho] = self.rho()
        [self.CarryRho] = self.carry_rho()
        [self.Vega] = self.vega()
        [self.Gamma] = self.gamma()
        [self.Phi] = self.phi()
        [self.Charm] = self.dDeltadTime()
        [self.Vanna] = self.dDeltadVol()

    def premium(self):
        tmpprem = self.Type * (self.S * e ** (-self.q * self.T) * norm.cdf(self.Type * self.d1) - \
                               self.K * e ** (-self.r * self.T) * norm.cdf(self.Type * self.d2))
        return [tmpprem]

    ############################################
    ############ 1st order greeks ##############
    ############################################

    def delta(self):
        dfq = e ** (-self.q * self.T)
        if self.Type == 1:
            return [dfq * norm.cdf(self.d1)]
        else:
            return [dfq * (norm.cdf(self.d1) - 1)]


    ########THESE ARE ORIGINAL FUNCTIONS FUNCTIONS IN USE ARE TO TRY AND IMPLEMENT FORWARD AND BACKWARD DIFFS#######
    #############################################################################################################
    # # Vega for 1% change in vol
    # def vega(self):
    #     return [0.01 * self.S * e ** (-self.q * self.T) * \
    #             norm.pdf(self.d1) * self.T ** 0.5]

    # # Theta for 1 day change
    # def theta(self):
    #     df = e ** -(self.r * self.T)
    #     dfq = e ** (-self.q * self.T)
    #     tmptheta = (1.0 / 365.0) \
    #                * (-0.5 * self.S * dfq * norm.pdf(self.d1) * \
    #                   self.sigma / (self.T ** 0.5) + \
    #                   self.Type * (self.q * self.S * dfq * norm.cdf(self.Type * self.d1) \
    #                                - self.r * self.K * df * norm.cdf(self.Type * self.d2)))
    #     return [tmptheta]

    def vega(self, h=0.0):
        return [0.01 * self.S * e ** (-self.q * (self.T - h)) * \
            norm.pdf(self.d1) * (self.T - h) ** 0.5]

    def theta(self, h=0.0):
        df = e ** -(self.r * (self.T - h))
        dfq = e ** (-self.q * (self.T - h))
        tmptheta = (1.0 / 365.0) \
                * (-0.5 * self.S * dfq * norm.pdf(self.d1) * \
                    self.sigma / ((self.T - h) ** 0.5) + \
                    self.Type * (self.q * self.S * dfq * norm.cdf(self.Type * self.d1) \
                                - self.r * self.K * df * norm.cdf(self.Type * self.d2)))
        return [tmptheta]

    def gamma(self, h=0.0):
        return [e ** (-self.q * (self.T - h)) * norm.pdf(self.d1) / (self.S * self.sigmaT)]


    def rho(self):
        df = e ** -(self.r * self.T)
        return [self.Type * self.K * self.T * df * 0.01 * norm.cdf(self.Type * self.d2)]
    
    from numpy import e

    def carry_rho(self):
        T = self.T
        S = self.S
        r = self.r
        q = self.q
        b = self.b
        d1 = self.d1
        d2 = self.d2
        df = e ** (-r * T)
        dfb = e ** ((b - r) * T)

        if self.Type == 1:  # Call
            carry_rho = T * S * dfb * norm.cdf(d1)
        else:  # Put
            carry_rho = -T * S * dfb * norm.cdf(-d1)

        return [carry_rho * 0.01]


    def phi(self):
        return [0.01 * -self.Type * self.T * self.S * \
                e ** (-self.q * self.T) * norm.cdf(self.Type * self.d1)]

    ############################################
    ############ 2nd order greeks ##############
    ############################################

    ########THESE ARE ORIGINAL FUNCTIONS FUNCTIONS IN USE ARE TO TRY AND IMPLEMENT FORWARD AND BACKWARD DIFFS#######
    #############################################################################################################
    # def gamma(self):
    #     return [e ** (-self.q * self.T) * norm.pdf(self.d1) / (self.S * self.sigmaT)]

    # Charm for 1 day change
    def dDeltadTime(self):
        dfq = e ** (-self.q * self.T)
        if self.Type == 1:
            return [
                (1.0 / 365.0) * -dfq * (norm.pdf(self.d1) * ((self.r - self.q) / (self.sigmaT) - self.d2 / (2 * self.T)) \
                                        + (-self.q) * norm.cdf(self.d1))]
        else:
            return [
                (1.0 / 365.0) * -dfq * (norm.pdf(self.d1) * ((self.r - self.q) / (self.sigmaT) - self.d2 / (2 * self.T)) \
                                        + self.q * norm.cdf(-self.d1))]

    # Vanna for 1% change in vol
    def dDeltadVol(self):
        return [0.01 * -e ** (-self.q * self.T) * self.d2 / self.sigma * norm.pdf(self.d1)]

    # Vomma
    def dVegadVol(self):
        return [0.01 * -e ** (-self.q * self.T) * self.d2 / self.sigma * norm.pdf(self.d1)]
    
    def approximate_forward_difference(self, func, h):
        fx = func()[0]

        # Modify the required variables
        if func == self.vega:
            self.sigma += h
        elif func == self.theta:
            self.T += h
        elif func == self.gamma:
            self.S += h

        # Recalculate the function value with the modified variables
        fx_plus_h = func()[0]

        # Restore the original variables
        if func == self.vega:
            self.sigma -= h
        elif func == self.theta:
            self.T -= h
        elif func == self.gamma:
            self.S -= h

        return (fx_plus_h - fx) / h

    def approximate_backward_difference(self, func, h):
        fx = func()[0]

        # Modify the required variables
        if func == self.vega:
            self.sigma -= h
        elif func == self.theta:
            self.T -= h
        elif func == self.gamma:
            self.S -= h

        # Recalculate the function value with the modified variables
        fx_minus_h = func()[0]

        # Restore the original variables
        if func == self.vega:
            self.sigma += h
        elif func == self.theta:
            self.T += h
        elif func == self.gamma:
            self.S += h

        return (fx - fx_minus_h) / h



    # Forward and backward difference approximations for Vega
    def vega_fd_approx(self, h=0.01):
        return self.approximate_forward_difference(self.vega, h)

    def vega_bd_approx(self, h=0.01):
        return self.approximate_backward_difference(self.vega, h)

    # Forward and backward difference approximations for Theta
    def theta_fd_approx(self, h=0.01):
        return self.approximate_forward_difference(self.theta, h)

    def theta_bd_approx(self, h=0.01):
        return self.approximate_backward_difference(self.theta, h)

    # Forward and backward difference approximations for Gamma
    def gamma_fd_approx(self, h=0.01):
        return self.approximate_forward_difference(self.gamma, h)

    def gamma_bd_approx(self, h=0.01):
        return self.approximate_backward_difference(self.gamma, h)
    
    def approximate_delta_fd(self, h=0.01):
        # Save the original underlying asset price
        original_S = self.S

        # Set the underlying asset price to the given value
        self.S += h

        # Calculate d1 and d2 with the new underlying asset price
        self.d1 = (log(self.S / self.K) + \
                   (self.r - self.q + 0.5 * (self.sigma ** 2)) \
                   * self.T) / self.sigmaT
        self.d2 = self.d1 - self.sigmaT

        # Calculate the option price
        option_price_plus_h = self.premium()[0]

        # Reset the underlying asset price to its original value
        self.S = original_S

        # Recalculate d1 and d2 with the original underlying asset price
        self.d1 = (log(self.S / self.K) + \
                   (self.r - self.q + 0.5 * (self.sigma ** 2)) \
                   * self.T) / self.sigmaT
        self.d2 = self.d1 - self.sigmaT

        # Calculate the option price
        option_price = self.premium()[0]

        # Calculate the approximate delta using forward differencing
        approximate_delta = (option_price_plus_h - option_price) / h

        return approximate_delta
    def approximate_delta_bd(self, h=0.01):
        # Save the original underlying asset price
        original_S = self.S

        # Set the underlying asset price to the given value
        self.S -= h

        # Calculate d1 and d2 with the new underlying asset price
        self.d1 = (log(self.S / self.K) + \
                (self.r - self.q + 0.5 * (self.sigma ** 2)) \
                * self.T) / self.sigmaT
        self.d2 = self.d1 - self.sigmaT

        # Calculate the option price
        option_price_minus_h = self.premium()[0]

        # Reset the underlying asset price to its original value
        self.S = original_S

        # Recalculate d1 and d2 with the original underlying asset price
        self.d1 = (log(self.S / self.K) + \
                (self.r - self.q + 0.5 * (self.sigma ** 2)) \
                * self.T) / self.sigmaT
        self.d2 = self.d1 - self.sigmaT

        # Calculate the option price
        option_price = self.premium()[0]

        # Calculate the approximate delta using backward differencing
        approximate_delta = (option_price - option_price_minus_h) / h

        return approximate_delta



In [3]:
import datetime as dt
from datetime import date
expiration = dt.datetime(2022,4,15)
current_date = dt.datetime(2022,3,13)

delta= expiration - current_date
delta.days

33

In [48]:
AAPLCall = [1, 151.03, 165, 0.0425, 0.0053, 33, 0.20]
bsm = BSMerton(AAPLCall)
vega_fd_approx = bsm.gamma_bd_approx(h=2)
print(vega_fd_approx)

# vega_bd_approx = bsm.vega_bd_approx(h=1)
# print(vega_bd_approx)


-0.00011288274912334834


In [49]:
AAPLCall = [1, 151.03, 165, 0.0425, 0.0053, 33, 0.20]
bsm = BSMerton(AAPLCall)
vega_fd_approx = bsm.vega_fd_approx(h=35)
vega_bd_approx = bsm.vega_bd_approx(h=35)
print(vega_fd_approx)
print(vega_bd_approx)


0.0
0.0


In [50]:
# For general accuracy purposes I looked up Implied volatility for AAPL back on 3/13/2022, it was 30.37%


AAPLCall = [1,151.03,165,.0425,.0053, 33,.20]
AAPLPut = [-1,151.03,165,.0425,.0053, 33,.2]

AAPL_Call_Premium = BSMerton(AAPLCall).Premium
AAPL_Call_Delta = BSMerton(AAPLCall).Delta
AAPL_Call_Gamma = BSMerton(AAPLCall).Gamma
AAPL_Call_Theta = BSMerton(AAPLCall).Theta
AAPL_Call_Vega = BSMerton(AAPLCall).Vega
AAPL_Call_Rho = BSMerton(AAPLCall).Rho
AAPL_Call_Rho_Carry = BSMerton(AAPLCall).CarryRho
AAPL_Call_Charm = BSMerton(AAPLCall).Charm
AAPL_Call_Vanna = BSMerton(AAPLCall).Vanna


AAPL_Call_Premium_Value = "AAPL_Call_Premium_Value"
AAPL_Call_Delta_Exposure = "AAPL Net Delta Exposure: "
AAPL_Call_Gamma_Exposure = "AAPL Net Gamma Exposure: "
AAPL_Call_Theta_Exposure = "AAPL Net Theta Exposure: "
AAPL_Call_Vega_Exposure = "AAPL Net Vega Exposure: "
AAPL_Call_Rho_Exposure = "AAPL Net Rho Exposure: "
AAPL_Call_Rho_Carry_Exposure = "AAPL Net Carry Rho Exposure: "
AAPL_Call_Charm_Exposure = "AAPL Net Charm Exposure: "
AAPL_Call_Vanna_Exposure = "AAPL Net Vanna Exposure: "

print(AAPL_Call_Premium_Value,AAPL_Call_Premium)
print(AAPL_Call_Delta_Exposure,AAPL_Call_Delta)
print(AAPL_Call_Gamma_Exposure,AAPL_Call_Gamma)
print(AAPL_Call_Theta_Exposure,AAPL_Call_Theta)
print(AAPL_Call_Vega_Exposure,AAPL_Call_Vega)
print(AAPL_Call_Rho_Exposure,AAPL_Call_Rho)
print(AAPL_Call_Rho_Carry_Exposure,AAPL_Call_Rho_Carry)
print(AAPL_Call_Charm_Exposure,AAPL_Call_Charm)
print(AAPL_Call_Vanna_Exposure,AAPL_Call_Vanna)


AAPL_Call_Premium_Value 0.3357989976315192
AAPL Net Delta Exposure:  0.08297130333914773
AAPL Net Gamma Exposure:  0.016822916101852648
AAPL Net Theta Exposure:  -0.022264444821010514
AAPL Net Vega Exposure:  0.06938710929513443
AAPL Net Rho Exposure:  0.01102593915636819
AAPL Net Carry Rho Exposure:  0.01132953825011723
AAPL Net Charm Exposure:  -0.003603543622813535
AAPL Net Vanna Exposure:  0.011041137391941661


In [57]:

AAPL_Put_Premium = BSMerton(AAPLPut).Premium
AAPL_Put_Delta = BSMerton(AAPLPut).Delta
AAPL_Put_Gamma = BSMerton(AAPLPut).Gamma
AAPL_Put_Theta = BSMerton(AAPLPut).Theta
AAPL_Put_Vega = BSMerton(AAPLPut).Vega
AAPL_Put_Rho = BSMerton(AAPLPut).Rho
AAPL_Put_Rho_Carry = BSMerton(AAPLPut).CarryRho
AAPL_Put_Charm = BSMerton(AAPLPut).Charm
AAPL_Put_Vanna = BSMerton(AAPLPut).Vanna

AAPL_Put_Premium_Value = "AAPL_Put_Premium_Value"
AAPL_Put_Delta_Exposure = "AAPL Net Delta Exposure: "
AAPL_Put_Gamma_Exposure = "AAPL Net Gamma Exposure: "
AAPL_Put_Theta_Exposure = "AAPL Net Theta Exposure: "
AAPL_Put_Vega_Exposure = "AAPL Net Vega Exposure: "
AAPL_Put_Rho_Exposure = "AAPL Net Rho Exposure: "
AAPL_Put_Rho_Carry_Exposure = "AAPL Net Carry Rho Exposure: "
AAPL_Put_Charm_Exposure = "AAPL Net Charm Exposure: "
AAPL_Put_Vanna_Exposure = "AAPL Net Vanna Exposure: "

print(AAPL_Put_Premium_Value,AAPL_Put_Premium)
print(AAPL_Put_Delta_Exposure,AAPL_Put_Delta)
print(AAPL_Put_Gamma_Exposure,AAPL_Put_Gamma)
print(AAPL_Put_Theta_Exposure,AAPL_Put_Theta)
print(AAPL_Put_Vega_Exposure,AAPL_Put_Vega)
print(AAPL_Put_Rho_Exposure,AAPL_Put_Rho)
print(AAPL_Put_Rho_Carry_Exposure,AAPL_Put_Rho_Carry)
print(AAPL_Put_Charm_Exposure,AAPL_Put_Charm)
print(AAPL_Put_Vanna_Exposure,AAPL_Put_Vanna)


AAPL_Put_Premium_Value 13.745361593880062
AAPL Net Delta Exposure:  -0.9165496333661425
AAPL Net Gamma Exposure:  0.016822916101852648
AAPL Net Theta Exposure:  -0.005317784872060155
AAPL Net Vega Exposure:  0.06938710929513443
AAPL Net Rho Exposure:  -0.1375800312273579
AAPL Net Carry Rho Exposure:  -0.1251527180054937
AAPL Net Charm Exposure:  -0.0036180572144972013
AAPL Net Vanna Exposure:  0.011041137391941661


In [93]:
from math import exp, sqrt
import numpy as np

def bt_american(call, underlying, strike, ttm, rf, b, ivol, N):
    dt = ttm / N
    u = exp(ivol * sqrt(dt))
    d = 1 / u
    pu = (exp(b * dt) - d) / (u - d)
    pd = 1.0 - pu
    df = exp(-rf * dt)
    z = 1 if call else -1

    def nNodeFunc(n):
        return int((n + 1) * (n + 2) // 2)

    def idxFunc(i, j):
        return nNodeFunc(j - 1) + i

    nNodes = nNodeFunc(N)
    optionValues = np.empty(nNodes)

    for j in range(N, -1, -1):
        for i in range(j, -1, -1):
            idx = idxFunc(i, j)
            price = underlying * u ** i * d ** (j - i)
            optionValues[idx] = max(0, z * (price - strike))

            if j < N:
                optionValues[idx] = max(optionValues[idx], df * (pu * optionValues[idxFunc(i + 1, j + 1)] + pd * optionValues[idxFunc(i, j + 1)]))

    return optionValues[0]


def binomial_tree_with_div(call, underlying, strike, ttm, rf, divAmts, divTimes, ivol, N):
    if not divAmts or not divTimes or divTimes[0] > N:
        return bt_american(call, underlying, strike, ttm, rf, rf, ivol, N)

    dt = ttm / N
    u = exp(ivol * sqrt(dt))
    d = 1 / u
    pu = (exp(rf * dt) - d) / (u - d)
    pd = 1.0 - pu
    df = exp(-rf * dt)
    z = 1 if call else -1

    def nNodeFunc(n):
        return int((n + 1) * (n + 2) // 2)

    def idxFunc(i, j):
        return nNodeFunc(j - 1) + i

    nDiv = len(divTimes)
    nNodes = nNodeFunc(N)
    optionValues = np.empty(nNodes)

    for j in range(int(divTimes[0]), -1, -1):
        for i in range(j, -1, -1):
            idx = idxFunc(i, j)
            price = underlying * u ** i * d ** (j - i)

            if j < divTimes[0]:
                optionValues[idx] = max(0, z * (price - strike))
                optionValues[idx] = max(optionValues[idx], df * (pu * optionValues[idxFunc(i + 1, j + 1)] + pd * optionValues[idxFunc(i, j + 1)]))
            else:
                valNoExercise = binomial_tree_with_div(call, price - divAmts[0], strike, ttm - divTimes[0] * dt, rf, divAmts[1:], [t - divTimes[0] for t in divTimes[1:]], ivol, N - divTimes[0])
                valExercise = max(0, z * (price - strike))
                optionValues[idx] = max(valNoExercise, valExercise)

    return optionValues[0]

call = True
put = False
underlying = 151.03
strike = 165
ttm = 33/365
rf = 0.0425
b = 0.0425
divAmts =  [0.88]
divTimes = [29/365]
ivol = 0.20
N = 300

# AAPL_option_call = binomial_tree_with_div(True, underlying, strike, ttm, rf, divAmts, divTimes, ivol, N)
AAPL_option_call_no_div = bt_american(call, underlying, strike, ttm, rf, b, ivol, N)
print(f"Option price call no div: {AAPL_option_call_no_div:.2f}")

AAPL_option_put_no_div = bt_american(put,underlying, strike, ttm, rf, b, ivol, N)
print(f"Option price put no div: {AAPL_option_put_no_div:.2f}")

AAPL_option_put = binomial_tree_with_div(put, underlying, strike, ttm, rf, divAmts, divTimes, ivol, N)
print(f"Option price put with div: {AAPL_option_put:.2f}")

AAPL_option_call = binomial_tree_with_div(call, underlying, strike, ttm, rf, divAmts, divTimes, ivol, N)
print(f"Option price call with div: {AAPL_option_call:.2f}")



Option price call no div: 0.34
Option price put no div: 14.02
Option price put with div: 13.97
Option price call with div: 6.77


In [94]:
def calculate_greeks(option_func, call, underlying, strike, ttm, rf, divAmts, divTimes, ivol, N):
    dS = 0.01 * underlying
    dV = 0.01 * ivol

    if option_func == bt_american:
        P = option_func(call, underlying, strike, ttm, rf, rf, ivol, N)
        P_up = option_func(call, underlying + dS, strike, ttm, rf, rf, ivol, N)
        P_down = option_func(call, underlying - dS, strike, ttm, rf, rf, ivol, N)
        P_vol_up = option_func(call, underlying, strike, ttm, rf, rf, ivol + dV, N)
        P_vol_down = option_func(call, underlying, strike, ttm, rf, rf, ivol - dV, N)
    else:
        P = option_func(call, underlying, strike, ttm, rf, divAmts, divTimes, ivol, N)
        P_up = option_func(call, underlying + dS, strike, ttm, rf, divAmts, divTimes, ivol, N)
        P_down = option_func(call, underlying - dS, strike, ttm, rf, divAmts, divTimes, ivol, N)
        P_vol_up = option_func(call, underlying, strike, ttm, rf, divAmts, divTimes, ivol + dV, N)
        P_vol_down = option_func(call, underlying, strike, ttm, rf, divAmts, divTimes, ivol - dV, N)

    delta = (P_up - P_down) / (2 * dS)
    gamma = (P_up - 2 * P + P_down) / (dS ** 2)
    vega = (P_vol_up - P_vol_down) / (2 * dV)

    return delta, gamma, vega


# For binomial_tree_with_div
delta, gamma, vega = calculate_greeks(binomial_tree_with_div, call, underlying, strike, ttm, rf, divAmts, divTimes, ivol, N)
print(f"Delta: {delta:.4f}\nGamma: {gamma:.4f}\nVega: {vega:.4f}")

print("\n")

# For bt_american
delta, gamma, vega = calculate_greeks(bt_american, call, underlying, strike, ttm, rf, [], [], ivol, N)
print(f"Delta: {delta:.4f}\nGamma: {gamma:.4f}\nVega: {vega:.4f}")

Delta: 0.0000
Gamma: 0.0000
Vega: -0.1834


Delta: 0.0854
Gamma: 0.0170
Vega: 7.3056


In [95]:
# Problem 3
import pandas as pd
import numpy as np
from datetime import datetime


In [100]:
# Load the CSV files
fama_french = pd.read_csv('F-F_Research_Data_Factors_daily.CSV')
momentum = pd.read_csv('F-F_Momentum_Factor_daily.CSV')

# Merge the datasets on the 'Date' column
factors = pd.merge(fama_french, momentum, on='Date')

# Convert the date to datetime format
factors['Date'] = pd.to_datetime(factors['Date'], format='%Y%m%d')

# Rename the 'Mom   ' column to 'Mom' (remove trailing spaces)
factors.rename(columns={'Mom   ': 'Mom'}, inplace=True)

# Divide factor values by 100 to match stock return units
factors[['Mkt-RF', 'SMB', 'HML', 'Mom']] = factors[['Mkt-RF', 'SMB', 'HML', 'Mom']] / 100

# Select the past 10 years of factor returns
mask = factors['Date'] >= (factors['Date'].max() - pd.DateOffset(years=10))
factors = factors[mask]


In [101]:
prices = pd.read_csv('DailyPrices.csv')
prices['Date'] = pd.to_datetime(prices['Date'], format='%m/%d/%Y %H:%M')
stocks = ['AAPL', 'META', 'UNH', 'MA', 'MSFT', 'NVDA', 'HD', 'PFE','AMZN','TSLA','GOOGL','BRK-B','JPM','JNJ','PG','V','DIS','BAC','XOM','CSCO']
returns = prices[stocks].pct_change().dropna()
returns['Date'] = prices['Date'][1:]

merged_returns = pd.merge(returns, factors, on='Date')

In [102]:
merged_returns



,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,TSLA,...,DIS,BAC,XOM,CSCO,Date,Mkt-RF,SMB,HML,RF,Mom
0,0.023152,0.015158,0.008073,0.019724,0.018542,0.091812,0.004836,-0.000201,0.008658,0.053291,...,0.025655,0.007803,-0.012535,0.020496,2022-02-15,0.0187,0.0133,-0.0142,0.000,-0.0091
1,-0.001389,-0.020181,0.003806,0.003643,-0.001167,0.000604,-0.008974,-0.002209,0.010159,0.001041,...,0.010535,-0.002302,0.004616,-0.000369,2022-02-16,-0.0002,-0.0009,0.0031,0.000,0.0064
2,-0.021269,-0.040778,-0.020227,-0.024103,-0.029282,-0.075591,-0.006141,-0.015700,-0.021809,-0.050943,...,-0.021746,-0.033767,-0.001531,0.028018,2022-02-17,-0.0228,-0.0028,0.0110,0.000,0.0103
3,-0.009356,-0.007462,-0.005379,-0.010035,-0.009631,-0.035296,-0.003075,-0.007566,-0.013262,-0.022103,...,-0.010396,-0.002388,-0.011121,0.025820,2022-02-18,-0.0087,-0.0009,0.0093,0.000,0.0104
4,-0.017812,-0.019790,-0.011329,-0.004487,-0.000729,-0.010659,-0.088506,-0.020606,-0.015753,-0.041366,...,-0.021604,-0.008703,-0.011634,-0.015906,2022-02-22,-0.0118,-0.0048,0.0011,0.000,0.0043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,-0.004701,-0.011457,0.001831,0.006263,-0.005908,0.003011,-0.001353,0.008052,0.008929,0.003753,...,0.020000,0.008678,-0.005272,0.003556,2023-01-25,0.0000,-0.0004,0.0065,0.017,0.0014
237,0.014803,0.040989,-0.000041,-0.013468,0.030714,0.024789,-0.010874,-0.009180,0.020992,0.109673,...,0.014613,0.013479,0.040191,0.007503,2023-01-26,0.0108,-0.0058,0.0001,0.017,-0.0123
238,0.013684,0.030143,-0.013056,-0.008509,0.000645,0.028431,0.009178,-0.010395,0.030437,0.110002,...,-0.001458,0.003113,-0.018257,0.003517,2023-01-27,0.0036,0.0062,-0.0116,0.017,-0.0246
239,-0.020078,-0.030842,-0.000535,-0.007780,-0.021962,-0.059072,-0.007736,-0.005481,-0.016530,-0.063182,...,-0.017802,-0.004231,-0.017732,-0.005978,2023-01-30,-0.0138,-0.0010,0.0072,0.017,0.0136


In [103]:
import statsmodels.api as sm

X = merged_returns[['Mkt-RF', 'SMB', 'HML', 'Mom']]
X = sm.add_constant(X)
regression_results = {}

for stock in stocks:
    Y = merged_returns[stock] - merged_returns['RF']
    model = sm.OLS(Y, X)
    result = model.fit()
    regression_results[stock] = result

expected_annual_returns = {}
for stock in stocks:
    result = sm.OLS(merged_returns[stock], X).fit()
    expected_return = np.dot(factors.mean().iloc[:-1], result.params[1:]) * 252
    expected_annual_returns[stock] = expected_return

expected_annual_returns = pd.Series(expected_annual_returns)


C:\Users\Moneybag Mike\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:16: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  app.launch_new_instance()


In [104]:
# Calculate the expected annual returns
expected_annual_returns = {}
for stock in stocks:
    results = sm.OLS(merged_returns[stock], X).fit()
    expected_return = np.dot(factors.mean().iloc[:-1], results.params[1:]) * 252
    expected_annual_returns[stock] = expected_return

expected_annual_returns = pd.Series(expected_annual_returns)


C:\Users\Moneybag Mike\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  """


In [105]:
expected_annual_returns

AAPL     0.245467
META    -0.360481
UNH      0.525284
MA       0.120099
MSFT     0.187455
NVDA     0.129642
HD       0.037879
PFE      0.251248
AMZN    -0.247582
TSLA     0.018265
GOOGL   -0.072408
BRK-B   -0.014605
JPM     -0.181540
JNJ      0.181391
PG       0.304027
V        0.097877
DIS     -0.289032
BAC     -0.205433
XOM      0.262903
CSCO     0.342819
dtype: float64

In [106]:
#create a covariance matrix

cov_matrix = returns[stocks].cov() * 252
print(cov_matrix)

           AAPL      META       UNH        MA      MSFT      NVDA        HD  \
AAPL   0.126877  0.139557  0.037447  0.081272  0.102937  0.171265  0.066193   
META   0.139557  0.400843  0.017102  0.102465  0.142255  0.240599  0.098845   
UNH    0.037447  0.017102  0.060922  0.031117  0.036318  0.046531  0.026045   
MA     0.081272  0.102465  0.031117  0.095762  0.079856  0.137369  0.056792   
MSFT   0.102937  0.142255  0.036318  0.079856  0.127839  0.175956  0.070916   
NVDA   0.171265  0.240599  0.046531  0.137369  0.175956  0.403814  0.112055   
HD     0.066193  0.098845  0.026045  0.056792  0.070916  0.112055  0.097074   
PFE    0.032745  0.045091  0.032068  0.033440  0.035082  0.046362  0.033271   
AMZN   0.122117  0.194794  0.034737  0.096194  0.133856  0.221364  0.096833   
TSLA   0.154880  0.173121  0.039128  0.097521  0.131911  0.291392  0.077754   
GOOGL  0.111906  0.182074  0.029806  0.079016  0.120259  0.188012  0.069626   
BRK-B  0.055520  0.061619  0.028173  0.047520  0.052

In [110]:
from scipy.optimize import minimize


np.set_printoptions(suppress=True, formatter={'float': '{:0.8f}'.format})

def portfolio_volatility(weights, cov_matrix):
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

def find_super_efficient_portfolio(expected_returns, cov_matrix, risk_free_rate):
    num_stocks = len(expected_returns)
    init_guess = np.repeat(1/num_stocks, num_stocks)
    bounds = ((0, 1),) * num_stocks
    weights_sum_to_1 = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}
    def neg_sharpe_ratio(weights, expected_returns, cov_matrix, risk_free_rate):
        portfolio_return = np.dot(weights, expected_returns)
        portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility
        return -sharpe_ratio

    optimization_result = minimize(neg_sharpe_ratio, init_guess,
                      args=(expected_returns.values, cov_matrix, risk_free_rate),
                      method='SLSQP', bounds=bounds, constraints=weights_sum_to_1)
    
    return optimization_result.x

    
risk_free_rate = 0.0425
super_efficient_weights = find_super_efficient_portfolio(expected_annual_returns, cov_matrix, risk_free_rate)

print("Super efficient portfolio weights:")
for stock, weight in zip(stocks, super_efficient_weights):
    print(f"{stock}: {weight:.8f}")



# risk_free_rate = 0.0425
# super_efficient_weights = find_super_efficient_portfolio(expected_annual_returns, cov_matrix, risk_free_rate)

# optimal_portfolio = pd.DataFrame(super_efficient_weights, index=stocks, columns=['Weight'])
# print(optimal_portfolio)


Super efficient portfolio weights:
AAPL: 0.00000000
META: 0.00000000
UNH: 0.82698586
MA: 0.00000000
MSFT: 0.00000000
NVDA: 0.00000000
HD: 0.00000000
PFE: 0.00000000
AMZN: 0.00000000
TSLA: 0.00000000
GOOGL: 0.00000000
BRK-B: 0.00000000
JPM: 0.00000000
JNJ: 0.00000000
PG: 0.07752476
V: 0.00000000
DIS: 0.00000000
BAC: 0.00000000
XOM: 0.00447762
CSCO: 0.09101176


In [111]:
# Calculate the expected return of the super efficient portfolio
portfolio_expected_return = np.dot(super_efficient_weights, expected_annual_returns)

# Calculate the portfolio volatility
portfolio_volatility = np.sqrt(np.dot(super_efficient_weights.T, np.dot(cov_matrix, super_efficient_weights)))

# Calculate the Sharpe ratio
sharpe_ratio = (portfolio_expected_return - risk_free_rate) / portfolio_volatility

print(f"Super efficient portfolio Sharpe ratio: {sharpe_ratio:.4f}")


Super efficient portfolio Sharpe ratio: 1.9776
